In [49]:

import requests
import time
import json

from tqdm import tqdm



# l1000 = []
# entrez_ids = []
# with open('../data/L1000.txt', 'r') as file:
#     for line in file:
#         name, entrez = line.rstrip().split('\t')
#         l1000.append(name.rstrip())
#         entrez_ids.append(entrez.rstrip())

# results = []
# for gene, entrez_id in zip(l1000, entrez_ids):
#     hgnc_fetch = f'https://rest.genenames.org/fetch/entrez_id/{entrez_id}'
#     response = requests.get(hgnc_fetch, headers={'Accept': 'application/json'})
#     response = response.json()
#     print((gene, entrez_id, response['response']['docs'][0]['uniprot_ids'][0]))

#     results.append((gene, entrez_id, response['response']['docs'][0]['uniprot_ids'][0]))
#     time.sleep(0.1)

# with open('../data/L1000_to_uniprot.txt', 'w') as f:
#     for gene_name, entrez_id, uniprot_assension in results:
#         f.write(gene_name + "\t" + entrez_id + "\t" + uniprot_assension + "\n")


# Download uniprot data
uniprot_data = []
with open('../data/L1000_to_uniprot.txt', 'r') as file:

    for line in tqdm(file):
        gene_name, entrez_id, uniprotkb_assension  = line.rstrip().split('\t')

        if uniprotkb_assension == '?':
            continue
        
        response = requests.get(f'https://rest.uniprot.org/uniprotkb/{uniprotkb_assension}.json')

        with open(f'../data/uniprot_data/by_id/{uniprotkb_assension}.json', 'w') as file:
            json.dump(response.json(), file)

        time.sleep(0.1)


1058it [08:15,  2.14it/s]


In [53]:
# store uniprot data in a csv file (L1000_name, uniprotkb_assension, function_description, subunit_interactions)
import pandas as pd

uniprot_data = []

with open('../data/L1000_to_uniprot.txt', 'r') as file:
    for line in file:
        gene_name, entrez_id, uniprotkb_assension  = line.rstrip().split('\t')

        if uniprotkb_assension == '?':
            continue

        with open(f'../data/uniprot_data/by_id/{uniprotkb_assension}.json', 'r') as f:
            protein_data = json.loads(f.read())

            for comment in protein_data['comments']:
                if comment['commentType'] == 'FUNCTION':
                    function_description = comment['texts'][0]['value']

                if comment['commentType'] == 'SUBUNIT':
                    subunit_interactions = comment['texts'][0]['value']

            uniprot_data.append((gene_name, uniprotkb_assension, function_description, subunit_interactions))

df = pd.DataFrame(uniprot_data, columns=['L1000_name', 'uniprotkb_assension', 'function_description', 'subunit_interactions'])
df.to_csv(f'../data/uniprot_data/uniprot_data.csv', index=False)